## THIS IS A SCRAP NOTEBOOK
In this note book you will find the code used to clean up the list of NYSE comapnies that were registered with the SEC 

In [1]:
# Importing needed libraries for data manipulation 
import numpy as np
import pandas as pd

from edgar import Company, TXTML, Edgar

from bs4 import BeautifulSoup

import re

import os

import ast

# Company Extraction 

In [ ]:
# List of companies to request information for
# The full list stems from the companies who report 10-Qs
cannabiscompanies = ['22nd Century Group Inc.', 'Altria Group, Inc.', 'Compass Diversified Holdings']

# In the same order as list above, this is the CIK number
# CIK_num = [0001347858]

# The years to pull the 10-Qs for
years = [2016, 2017, 2018, 2019, 2020]

In [ ]:
company1 = Company("22nd Century Group Inc.", "0001347858")
tree1 = company1.get_all_filings(filing_type = "10-Q")
docs1 = Company.get_documents(tree1, no_of_documents=13)



In [ ]:
example1 = docs1[0]

In [ ]:
#  print(example1.text_content())

In [67]:
# # /Users/boimoriba/Documents/Learn.Co_Docs/Projects/Capstone/CSV_Files/cik-lookup-data.txt

# THIS_FOLDER = os.path.dirname(os.path.abspath(__file__))
# my_file = os.path.join(THIS_FOLDER, "cik-lookup-data.txt")
# File_object = open("cik-lookup-data.txt","r")

In [6]:
f = open('/Users/boimoriba/Documents/Learn.Co_Docs/Projects/Capstone/Notebooks/cik-lookup-data.txt')

In [12]:
SEC_REG_list = pd.read_fwf('cik-lookup-data.txt', colspecs='infer', widths=None, )

In [13]:
SEC_REG_list.head()

,!J INC:0001438823:
0,"#1 A LIFESAFER HOLDINGS, INC.:0001509607:"
1,#1 ARIZONA DISCOUNT PROPERTIES LLC:0001457512:
2,#1 PAINTBALL CORP:0001433777:
3,$ LLC:0001427189:
4,"$AVY, INC.:0001655250:"


In [14]:
SEC_REG_list.columns

Index(['!J INC:0001438823:'], dtype='object')

In [15]:
SEC_REG_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 759377 entries, 0 to 759376
Data columns (total 1 columns):
!J INC:0001438823:    759377 non-null object
dtypes: object(1)
memory usage: 5.8+ MB


In [16]:
SEC_REG_list.shape

(759377, 1)

In [21]:
type(SEC_REG_list['!J INC:0001438823:'][1])

str

In [24]:
SEC_REG_list['!J INC:0001438823:'][700]

'12 WEST CAPITAL MANAGEMENT LP:0001540531:'

need to split by : and  create cik_number column 

then drop : in front of cik number in cik_number column 

In [63]:
# test run
txt = SEC_REG_list['!J INC:0001438823:'][700]

x = txt.split(":")
x

['12 WEST CAPITAL MANAGEMENT LP', '0001540531', '']

In [55]:
c_columns = SEC_REG_list['!J INC:0001438823:']

companies= []     
for comp in c_columns:
    companies.append(comp.split(":"))

    

In [151]:
companies[759376][0]

'ZZZ MEDIA HOLDINGS, INC.'

In [143]:
len(companies)

759377

In [155]:
# name = []
# cik = []

# name_dict={}
# cik_dict={}

# for i in range(0,100) :
#     name = companies[i][0]
#     cik = companies[i][1]
    
#     name_dict[i]=name
#     cik_dict[i]=cik
    


In [191]:
# cik_dict

In [135]:
top10 = ['3M',
'Abbott Labs',
'Accenture',
'AIG',
'Altria',
'AT&T',
'Baker Hughes A',
'Bank of America',
'Bank of NY Mellon',
'Baxter']

In [136]:
df = pd.DataFrame(companies, columns = ['Name', 'CIK_Num', 'Space', 'Space1', 'Space3'])  

In [137]:
df.head()

,Name,CIK_Num,Space,Space1,Space3
0,"#1 A LIFESAFER HOLDINGS, INC.",0001509607,,None,None
1,#1 ARIZONA DISCOUNT PROPERTIES LLC,0001457512,,None,None
2,#1 PAINTBALL CORP,0001433777,,None,None
3,$ LLC,0001427189,,None,None
4,"$AVY, INC.",0001655250,,None,None


In [183]:
df.tail()

,Name,CIK_Num,Space,Space1,Space3
759372,ZZIF 2008 INVESTMENT LLC,0001448632,,None,None
759373,"ZZLL INFORMATION TECHNOLOGY, INC",0001365357,,None,None
759374,"ZZX, LLC",0001691924,,None,None
759375,ZZYZX ZZAZX ZZOZX INC,0001184274,,None,None
759376,"ZZZ MEDIA HOLDINGS, INC.",0001755250,,None,None


In [138]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 759377 entries, 0 to 759376
Data columns (total 5 columns):
Name       759377 non-null object
CIK_Num    754640 non-null object
Space      729181 non-null object
Space1     269 non-null object
Space3     2 non-null object
dtypes: object(5)
memory usage: 29.0+ MB


In [193]:
df.shape

(759377, 5)

# Space 3

In [204]:
df['Space3'].isna().sum()

759375

In [201]:
df['Space3'].value_counts()

    2
Name: Space3, dtype: int64

In [207]:
df.loc[df['Space3'] == '']

,Name,CIK_Num,Space,Space1,Space3
154781,CONNECT 1,2,1 MOBILE INC,0001361165,
655695,STUDIO,,J'HAN CO LLC,0001734395,


# Space 1 Column
Need to figure what is in this column, it may be a cik number so I want to make sure I check it before I drop anything from this dataframe. 

In [198]:
df.loc[df['Space1'] == ''] 

,Name,CIK_Num,Space,Space1,Space3
659,11,11 CAPITAL CORP.,0001463262,,None
3032,29,11 PLAN LLC,0001738803,,None
3076,2I CAPITAL PCC,STRATEGIC FUND I,0001508295,,None
3730,3,"10 CAPITAL WPF II, LLC",0001728252,,None
3731,3,"10 CAPITAL WPF III, LLC",0001734857,,None
...,...,...,...,...,...
731110,WELLINGTON,NON-US EQUITY OFFSHORE LP,0001410166,,None
731111,WELLINGTON,SMALL CAP CORE OFFSHORE LP,0001359228,,None
739658,WILLIAM BLAIR,NON-US EQUITY LLC,0001359241,,None
739659,WILLIAM BLAIR,NON-US EQUITY OFFSHORE LP,0001359245,,None


In [197]:
df['Space1'].value_counts()

              267
0001361165      1
0001734395      1
Name: Space1, dtype: int64

In [199]:
df.loc[df['Space1'] == '0001734395']

,Name,CIK_Num,Space,Space1,Space3
655695,STUDIO,,J'HAN CO LLC,0001734395,


In [200]:
df.loc[df['Space1'] == '0001361165'] 

,Name,CIK_Num,Space,Space1,Space3
154781,CONNECT 1,2,1 MOBILE INC,0001361165,


# Space Column

In [212]:
df.loc[df['Space'] == ''] 

,Name,CIK_Num,Space,Space1,Space3
0,"#1 A LIFESAFER HOLDINGS, INC.",0001509607,,None,None
1,#1 ARIZONA DISCOUNT PROPERTIES LLC,0001457512,,None,None
2,#1 PAINTBALL CORP,0001433777,,None,None
3,$ LLC,0001427189,,None,None
4,"$AVY, INC.",0001655250,,None,None
...,...,...,...,...,...
759372,ZZIF 2008 INVESTMENT LLC,0001448632,,None,None
759373,"ZZLL INFORMATION TECHNOLOGY, INC",0001365357,,None,None
759374,"ZZX, LLC",0001691924,,None,None
759375,ZZYZX ZZAZX ZZOZX INC,0001184274,,None,None


# CIK_Num Column

In [222]:
df.iloc[6862]['Name']

'A-FTB-15-FUND, A SERIES OF HERON ROCK, ENDEAVOR NORTH FUND, LP'

In [219]:
df.loc[df['CIK_Num'] == '']

,Name,CIK_Num,Space,Space1,Space3
3594,36ONE HEDGE SEGREGATED PORTFOLIO OF THE SOUTH ...,,None,None,None
6716,"A-DHT-28-FUND, A SERIES OF HERON ROCK, ENDEAVO...",,None,None,None
6862,"A-FTB-15-FUND, A SERIES OF HERON ROCK, ENDEAVO...",,None,None,None
6892,"A-GIM-15-FUND, A SERIES OF HERON ROCK, ENDEAVO...",,None,None,None
6929,"A-HCD-15-FUND, A SERIES OF HERON ROCK, ENDEAVO...",,None,None,None
...,...,...,...,...,...
744715,WNC INSTITUTIONAL TAX CREDIT FUND X CALIFORNIA...,,None,None,None
746241,WOODBURY FINANCIAL SERVICES INC\\\\\\MN ...,,None,None,None
747543,WORLDQUANT MILLENNIUM WMQS GEAE OFFSHORE FUND ...,,None,None,None
757802,ZILKER MULTI-STRATEGIES INSURANCE DEDICATED FU...,,None,None,None


In [225]:
df2 = df.dropna(subset=['CIK_Num'],)

In [226]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 754640 entries, 0 to 759376
Data columns (total 5 columns):
Name       754640 non-null object
CIK_Num    754640 non-null object
Space      729181 non-null object
Space1     269 non-null object
Space3     2 non-null object
dtypes: object(5)
memory usage: 34.5+ MB


In [230]:
df3 = df2.drop(columns=['Space1', 'Space3', 'Space'])

In [231]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 754640 entries, 0 to 759376
Data columns (total 2 columns):
Name       754640 non-null object
CIK_Num    754640 non-null object
dtypes: object(2)
memory usage: 17.3+ MB


In [233]:
df3.isna().sum()

Name       0
CIK_Num    0
dtype: int64

In [253]:
df3.Name = df3.Name.str.title()

In [408]:
df3.head()

,Name,CIK_Num
0,"#1 A Lifesafer Holdings, Inc.",0001509607
1,#1 Arizona Discount Properties Llc,0001457512
2,#1 Paintball Corp,0001433777
3,$ Llc,0001427189
4,"$Avy, Inc.",0001655250


In [413]:
NYSE_US

['3M Co',
 'Abbott Laboratories',
 'Accenture Plc',
 'Allegiant Travel Co',
 'American International Group Inc.',
 'Baker Hughes Co',
 'Baxter International Inc',
 'Boeing Co',
 'Citigroup',
 'Devon Energy Corp',
 'Duke Energy',
 'Goldman Sachs Group Inc',
 'Home Depot',
 'Johnson & Johnson',
 'Kimberly-Clark',
 'Mastercard Inc',
 'MetLife',
 'Morgan Stanley',
 'Nike',
 'Philip Morris International Inc.',
 'Procter&Gamble',
 'Prudential Financial Inc',
 'Raytheon Technologies Corp',
 'Simon Property Group Inc',
 'Skywest Inc',
 'Southern Co']

In [421]:
NYSE_US = ['3M Co',
'Abbott Laboratories',
'Accenture Plc',
'Allegiant Travel Co',           
'American International Group Inc.',
'Baker Hughes Co',
'Baxter International Inc',
'Boeing Co',
'Citigroup',
'Devon Energy Corp',
'Duke Energy',
'Goldman Sachs Group Inc',
'Home Depot',
'Johnson & Johnson',
'Kimberly-Clark',
'Mastercard Inc',
'MetLife',
'Morgan Stanley',
'Nike',
'Philip Morris International Inc.',
'Procter&Gamble',
'Prudential Financial Inc',
'Raytheon Technologies Corp',
'Simon Property Group Inc',
'Skywest Inc',
'Southern Co']

In [430]:
# df3.isin.loc[df3['Name'] NYSE_US]
df_NYSE = df3[df3['Name'].isin(NYSE_US)]
df_NYSE = df_NYSE.to_csv('SEC Registered NYSE US Companies Down Sampled')

In [427]:
df_NYSE

,Name,CIK_Num
3931,3M Co,0000066740
9138,Abbott Laboratories,0000001800
11358,Accenture Plc,0001467373
27062,Allegiant Travel Co,0001362468
61458,Baker Hughes Co,0001701605
68424,Baxter International Inc,0000010456
90064,Boeing Co,0000012927
188508,Devon Energy Corp,0001090012
284701,Goldman Sachs Group Inc,0000886982
284702,Goldman Sachs Group Inc,0000904571


In [429]:
df_NYSE.shape

(19, 2)

In [404]:
# def NYSE(list_of_names):
#     for name in NYSE_US:
#         Names = df3[df3['Name'].str.contains(name)]
#         for cik in NYSE_US:
#             CIKs = df3[df3['CIK_Num'].str.contains(cik)]
#             return Names, CIKs 

In [405]:
# NYSE(NYSE_US)

(Empty DataFrame
 Columns: [Name, CIK_Num]
 Index: [], Empty DataFrame
 Columns: [Name, CIK_Num]
 Index: [])

In [390]:
# df3[df3['Name'].str.contains(NYSE_US)]

In [406]:
# df3.to_csv('List of SEC Registered NYSE US Companies')

# This is the test area of the imported methods from edgar library

In [ ]:
# company1 = Company("22nd Century Group Inc.", "0001347858")
# tree1 = company1.get_all_filings(filing_type = "10-Q", prior_to= 2016)
# docs1 = Company.get_documents(tree1, no_of_documents=13)


In [356]:
# import time
# time.sleep(1)  # delays for 1 second
# time.sleep(5)  # delays for 5 seconds
# time.sleep(60)  # delays for 1 minute

In [369]:
# for company in NYSE_US:
#     for cik in df3['CIK_Num']:
#         company1 = Company(company, cik)
#         tree1 = company1.get_all_filings(filing_type = "10-Q", prior_to= "2016")
#         docs1 = Company.get_documents(tree1, no_of_documents=1)
        

In [ ]:
# edgar.

In [351]:
# edgar = Edgar()
# possible_companies = edgar.get_cik_by_company_name('')

In [352]:
# possible_companies

In [296]:
# possible_companies = edgar.find_company_name("General Mills, Inc.")

In [192]:
# possible_companies

# Resources

**The Idea**

The Blog That Led to This Project: https://towardsdatascience.com/useful-sentiment-analysis-mining-sec-filings-part-1-358942fc98ed


**Edgar**

Edgar Documentation: https://pypi.org/project/edgar/


Video on How to Install, Import, and Request via Edgar Library: https://www.youtube.com/watch?v=_eN7D0IKu4Q

Video on How to Use SEC.gov Website to Parse Data: https://www.youtube.com/watch?v=4PMEBk9bZSo



**Company List**

List of SEC Registered Companies: https://www.sec.gov/Archives/edgar/cik-lookup-data.txt


Top NYSE US Companies: https://www.investing.com/indices/nyse-us-100-components


**Pandas Cheat Sheet**

https://morphocode.com/pandas-cheat-sheet/

